# Using the Qobuz API (`minim.qobuz`)

**Last updated**: June 6, 2023

In [1]:
import os
import pathlib
import sys

import ipywidgets

sys.path.insert(0, f"{pathlib.Path(os.path.abspath('')).parents[2].resolve()}/src")
# The second argument above should be replaced with the path where you downloaded Minim.
from minim import qobuz

## Getting started

The `minim.qobuz` submodule offers procedural and object-oriented approaches to use the Qobuz API:

* The `minim.qobuz.Session` class, which is targeted at developers and experienced programmers, implements the Qobuz API exactly on top of the `requests` library. There are no bells and whistles; responses from API calls, if any, are in JavaScript Object Notation (JSON) format, and data extraction from or wrangling of the JSON data is done solely by the user.

* The `minim.qobuz.Album`, `minim.qobuz.Artist`, `minim.qobuz.Label`, `minim.qobuz.Playlist`, `minim.qobuz.Track`, and `minim.qobuz.User` classes are more user-friendly and are meant for beginners or hobbyists. Instead of having the user make API calls directly and deal with unpacking the JSON responses, these classes do all the heavy lifting and store the information returned by the API in instance attributes that have simple, logical names and are well-documented.

Regardless of which approach above you choose, it is recommended that you have a Qobuz account and, preferably, a Qobuz streaming plan to get access to all the API endpoints.

## As a first example

To compare the procedural and object-oriented approaches and as a first introductory example, let's search for and output basic information about the track "Post Malone (feat. RANI)" by Sam Feldt.

### Procedural approach

First, we create a `qobuz.Session` object without user authentication:

In [2]:
session = qobuz.Session(authenticate=False)
print(session)

Qobuz API: not logged in


As expected, the string representation of the `qobuz.Session` object confirms that no user authentication was performed. A consequence is that certain API endpoints are unavailable since they require either the user to be logged in or have an active Qobuz streaming plan.

Then, we search for tracks (or releases) with titles containing the phrase "Post Malone". For brevity, we limit our search results to only two items. The search results are in JSON format, with the first layer specifying the media type (`"albums"`, `"artists"`, `"tracks"`, etc.) and the second the number of results for that media type and Qobuz catalog information for each match. We can preview the raw data for each track by iterating:

In [3]:
search_results = session.search("Post Malone", "track", strict=True, limit=2)
for track in search_results["tracks"]["items"]:
    print(track)

{'maximum_bit_depth': 16, 'copyright': '© 2019 SpinninRecords.com ℗ 2019 SpinninRecords.com', 'performers': 'Rani, FeaturedArtist - Roxanne Emery, Writer - Jordan Schultz, Masterer - Sam Feldt, Programmer, MainArtist - Sammy Renders, Writer - Dominic Lyttle, Mixer, Writer - Jonas Kröper, Writer - Shannon Hilversum, Writer', 'audio_info': {'replaygain_track_peak': 1, 'replaygain_track_gain': -10.44}, 'performer': {'name': 'Sam Feldt', 'id': 1829176}, 'album': {'image': {'small': 'https://static.qobuz.com/images/covers/cc/09/tfjs3torp09cc_230.jpg', 'thumbnail': 'https://static.qobuz.com/images/covers/cc/09/tfjs3torp09cc_50.jpg', 'large': 'https://static.qobuz.com/images/covers/cc/09/tfjs3torp09cc_600.jpg'}, 'maximum_bit_depth': 16, 'media_count': 1, 'artist': {'image': None, 'name': 'Sam Feldt', 'id': 1829176, 'albums_count': 245, 'slug': 'sam-feldt', 'picture': None}, 'upc': '0190295418267', 'released_at': 1558648800, 'label': {'name': "Spinnin' Records", 'id': 190680, 'albums_count': 4

The first search result is the track we are looking for. We can extract the key details of the track by traversing through the `dict` containing the Qobuz catalog information for that track:

In [4]:
track = search_results["tracks"]["items"][0]
print(f"Title: {track['title']}")
print(f"Artist: {track['performer']['name']}")
print(f"Album: {track['album']['title']}")
print(f"Genre: {track['album']['genre']['name']}")
print(f"Release date: {track['streamable_at']}")

Title: Post Malone (feat. RANI)
Artist: Sam Feldt
Album: Magnets EP
Genre: Dance
Release date: 1685170800


While most data in the `dict` is human-readable, certain fields, such as those dealing with datetimes, may require additional processing to be understandable. For example, the release date above is presented as an `int` representing the number of seconds since the Unix epoch, and must be converted to a datetime using `datetime.datetime.utcfromtimestamp()`.

Now, let's get the producing and songwriting credits for the track:

In [5]:
credits = session.get_track_credits(performers=track["performers"])
for role, artists in credits.items():
    print(f"{' '.join(role.split('_')).title()}(s): {', '.join(artists)}")

Programmer(s): Sam Feldt
Mixer(s): Dominic Lyttle
Featured Artist(s): Rani
Writer(s): Roxanne Emery, Sammy Renders, Dominic Lyttle, Jonas Kröper, Shannon Hilversum
Main Artist(s): Sam Feldt
Masterer(s): Jordan Schultz


What if we are building an application or widget to stream audio? We can get the URL of the track's stream using the track's Qobuz ID:

In [6]:
stream_url = session.get_track_file_url(track["id"])["url"]
ipywidgets.Audio.from_url(stream_url, autoplay=False)

/mnt/c/Users/Benjamin/Documents/GitHub/minim/src/minim/qobuz.py:912: UserWarning: No user authentication or Qobuz streaming plan detected. The URL will lead to a 30-second preview of the track.
  warnings.warn(wmsg)


Audio(value=b'https://streaming-qobuz-std.akamaized.ne...', autoplay='False', format='url')

Whoops! Because we didn't log into a Qobuz account with an active Qobuz streaming plan previously, we only got the 30-second preview of the track instead of the full audio. Let's authenticate ourselves and try again. 

**Note**: No parameters are passed to `qobuz.Session.login()` below because an Qobuz user authentication token was found in my operating system's environment variable `QOBUZ_USER_AUTH_TOKEN`. If you have not stored user credentials or an authorization token in your OS's environment variables, you can pass either the `auth_token` argument or the `email` and `password` arguments to `qobuz.Session.login()`.

In [7]:
session.login()
stream_url = session.get_track_file_url(track["id"])["url"]
# ipywidgets.Audio.from_url(stream_url, autoplay=False) 
# The line above is commented out for copyright reasons.

**Note**: We can also get the track's audio stream directly as a `bytes` object. See the [Working with tracks](#working-with-tracks) section below for more information.

### Object-oriented approach

First, we create a `qobuz.User` object without user authentication:

In [8]:
user = qobuz.User(authenticate=False)
print(user)

Qobuz user: not logged in


As expected, the string representation of the `qobuz.User` object confirms that no user authentication was performed.

**Note**: All objects spawned by the `qobuz.User` object shares the same Qobuz session information. Therefore, a `qobuz.Track` object created by the `qobuz.User` object above will also not be able to access information that requires user authentication. However, should the `qobuz.User` object be authenticated via `qobuz.User.login()`, the `qobuz.Track` object would also be authenticated.

Like before, we search for tracks with titles containing the phrase "Post Malone", but now limit our search results to only five items. The search results are still contained in a `dict`, with the first key specifying the media type (`"albums"`, `"artists"`, `"tracks"`, etc.). However, the value corresponding to the `"tracks"` key is now a `list` of `minim.qobuz.Track` objects, each fully populated with that track's information.

In [9]:
search_results = user.search("Post Malone", "track", strict=True, limit=5)
for track in search_results["tracks"]:
    print(track)

Qobuz track: 'Post Malone (feat. RANI)' [artist: Sam Feldt, album: Magnets EP]
Qobuz track: 'Post Malone' [artist: Rowan Duy, album: Night Changes]
Qobuz track: 'Post Malone (feat. RANI)' [artist: Sam Feldt, album: Post Malone (feat. RANI) [Joe Stone Remix]]
Qobuz track: 'Post Malone (feat. RANI) [VIZE Remix]' [artist: Sam Feldt, album: Post Malone (feat. RANI) [VIZE Remix]]
Qobuz track: 'Post Malone (feat. RANI) [GATTÜSO Remix]' [artist: Sam Feldt, album: Post Malone (feat. RANI) [The Remixes]]


With human-readable string representations of each track, it is easy to see that the first search result is the track we are looking for. We can extract the key details of the track by getting instance attributes of the `minim.qobuz.Track` object:

In [10]:
track = search_results["tracks"][0]
print(f"Title: {track.title}")
print(f"Artist: {track.main_artist.name}")
print(f"Album: {track.album.title}")
print(f"Genre: {track.album.genre}")
print(f"Release date: {track.release_date}")

Title: Post Malone (feat. RANI)
Artist: Sam Feldt
Album: Magnets EP
Genre: Dance
Release date: 2023-05-27 07:00:00


Now, all data in the `minim.qobuz.Track` object is human-readable, even those dealing with datetimes.

Furthermore, the producing and songwriting credits for the track are already embedded in the `minim.qobuz.Track` object:

In [11]:
for role, artists in track.credits.items():
    print(f"{' '.join(role.split('_')).title()}(s): {', '.join(artists)}")

Programmer(s): Sam Feldt
Mixer(s): Dominic Lyttle
Featured Artist(s): Rani
Writer(s): Roxanne Emery, Sammy Renders, Dominic Lyttle, Jonas Kröper, Shannon Hilversum
Main Artist(s): Sam Feldt
Masterer(s): Jordan Schultz


Getting the audio stream data no longer requires knowledge of the track ID:

In [12]:
track.get_file_url()
ipywidgets.Audio.from_url(track.file_url, autoplay=False)

Audio(value=b'https://streaming-qobuz-std.akamaized.ne...', autoplay='False', format='url')

Oops, forgot about user authentication again. This time, we authenticate using `minim.Qobuz.User.login()` instead:

In [13]:
user.login()
track.get_file_url()
# ipywidgets.Audio.from_url(track.file_url, autoplay=False)
# The line above is commented out for copyright reasons.

## Working with artists

An artist is a person or group that performs, appears on, or contributed to an album or track. Every album, playlist, and track contains artist information in some form.

In the following section, we will retrieve information about an artist (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will focus on the Dutch EDM duo Sick Individuals.

In [14]:
artist_id = 764370

### Procedural approach

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the artist. To include albums, playlists, and tracks that the artist appears on, we pass `extras=["albums", "playlists", "tracks"]` to `qobuz.Session.get_artist()`.

In [15]:
session = qobuz.Session(authenticate=False)
artist = session.get_artist(artist_id, extras=["albums", "playlists", "tracks"], limit=5)
for key, value in artist.items():
    print(key, value, sep=": ")

id: 764370
name: Sick Individuals
albums_as_primary_artist_count: 148
albums_as_primary_composer_count: 0
albums_count: 121
slug: sick-individuals
picture: None
image: {'small': 'https://static.qobuz.com/images/artists/covers/small/7e0606708c0ef65462c5311fe81099de.jpg', 'medium': 'https://static.qobuz.com/images/artists/covers/medium/7e0606708c0ef65462c5311fe81099de.jpg', 'large': 'https://static.qobuz.com/images/artists/covers/large/7e0606708c0ef65462c5311fe81099de.jpg', 'extralarge': 'https://static.qobuz.com/images/artists/covers/large/7e0606708c0ef65462c5311fe81099de.jpg', 'mega': 'https://static.qobuz.com/images/artists/covers/large/7e0606708c0ef65462c5311fe81099de.jpg'}
similar_artist_ids: [67661, 97536, 725598, 2103126, 301564, 720916, 955835, 1171943, 798810, 974660, 1048406]
information: None
biography: {'summary': 'Dutch EDM duo Sick Individuals write boisterous, catchy dance tunes equally suited for festivals and the radio. The duo consists of Joep Smeele and Rinze Hofstee, 

We can see information about the albums, playlists, and tracks stored in their respective keys above.

A biography for or introduction to the artist is outputted below:

In [16]:
print(artist['biography']['content'].split('\n')[0])

Dutch EDM duo Sick Individuals write boisterous, catchy dance tunes equally suited for festivals and the radio. The duo consists of Joep Smeele and Rinze Hofstee, who met in 2008 while studying music composition. Both individuals already had experience penning music for commercials, but they bonded over a mutual love for acts like Daft Punk and Swedish House Mafia, and decided to make dance music together. Sick Individuals began releasing original tracks on Sneakerz Muzik in 2010, and were initially more prolific as remixers, with clients including Flo Rida, Nicky Romero, and Freestylers. Their mix of Icona Pop's 2012 smash hit "I Love It" was particularly well received, and the duo followed it with numerous singles on labels like Revealed Recordings, Axtone, Onelove, and Armada. Collaborations with Dannic ("Blueprint") and Axwell ("I Am") both topped charts of dance music retailer Beatport, and the duo remixed songs by Madonna, Rihanna, Lady Gaga, Avicii, and numerous other stars. Sic

### Object-oriented approach

First, we directly create a `qobuz.Artist` object using the artist's Qobuz ID. We also request albums, playlists, and tracks that the artist contributed to by including the `extras=["albums", "playlists", "tracks"]` parameter. Information about the artist is automatically retrieved.

In [17]:
artist = qobuz.Artist(artist_id, extras=["albums", "playlists", "tracks"], limit=5, authenticate=False)
print(artist)

Qobuz artist: Sick Individuals


Below are the albums, playlists, and tracks that were requested:

In [18]:
for album in artist.albums:
    print(album)
for playlist in artist.playlists:
    print(playlist)
for track in artist.tracks:
    print(track)

Qobuz album: 'We Go Out' [artist: Alesso]
Qobuz album: 'My Life (feat. The Runaway Club)' [artist: THE RUNAWAY CLUB]
Qobuz album: 'With My Friends' [artist: Tungevaag]
Qobuz album: 'Miss You' [artist: Tungevaag]
Qobuz album: 'Feel Your Love' [artist: Dannic]
Qobuz playlist: 'Sick Individuals' [owner: Qobuz]
Qobuz track: 'MAIA' [artist: Sick Individuals, album: MAIA]
Qobuz track: 'Heart On Fire' [artist: Sick Individuals, album: Heart On Fire]
Qobuz track: 'YOU (feat. Dotter)' [artist: Sick Individuals, album: YOU (feat. Dotter)]
Qobuz track: 'My Life (feat. The Runaway Club)' [artist: Sick Individuals, album: My Life (feat. The Runaway Club)]
Qobuz track: 'Better with You' [artist: Sick Individuals, album: Better with You]


Like before, a brief summary of the artist is available:

In [19]:
print(artist.biography.split("\n")[0])

Dutch EDM duo Sick Individuals write boisterous, catchy dance tunes equally suited for festivals and the radio. The duo consists of Joep Smeele and Rinze Hofstee, who met in 2008 while studying music composition. Both individuals already had experience penning music for commercials, but they bonded over a mutual love for acts like Daft Punk and Swedish House Mafia, and decided to make dance music together. Sick Individuals began releasing original tracks on Sneakerz Muzik in 2010, and were initially more prolific as remixers, with clients including Flo Rida, Nicky Romero, and Freestylers. Their mix of Icona Pop's 2012 smash hit "I Love It" was particularly well received, and the duo followed it with numerous singles on labels like Revealed Recordings, Axtone, Onelove, and Armada. Collaborations with Dannic ("Blueprint") and Axwell ("I Am") both topped charts of dance music retailer Beatport, and the duo remixed songs by Madonna, Rihanna, Lady Gaga, Avicii, and numerous other stars. Sic

## Working with tracks

A track (or a release) is the most central Qobuz object. Tracks are a key part of albums and playlists and can be associated with artists who appear or feature on them.

In the following section, we will retrieve information about a track (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will use the track 'Runaway (U & I)' by Galantis.

In [20]:
track_id = 23929516

### Procedural approach

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the track:

In [21]:
session = qobuz.Session(authenticate=False)
track = session.get_track(track_id)
for key, value in track.items():
    print(key, value, sep=": ")

maximum_bit_depth: 24
copyright: © 2015 Atlantic Recording Corporation for the United States and WEA International for the world outside of the United States. A Warner Music Group Company ℗ 2014 Atlantic Recording Corporation
performers: Cathy Dennis, Lead Vocals - Kevin Grainger, MasteringEngineer - Christian Karlsson, Writer - Niklas Flyckt, MixingEngineer - Linus Eklöw, Writer - Julia Karlsson, Lead Vocals, Writer - Catherine Dennis, Writer - Galantis, MainArtist - Jimmy Koitzsch, Writer - Christian 'Bloodshy' Karlsson, Producer, Instruments, MixingEngineer - Anton Rundberg, Programmer, Writer - Jimmy 'Svidden' Koitzsch, Producer - Linus 'Style Of Eye' Eklöw, Producer, Instruments, MixingEngineer
audio_info: {'replaygain_track_gain': -10.91, 'replaygain_track_peak': 1}
performer: {'id': 865362, 'name': 'Galantis'}
album: {'maximum_bit_depth': 24, 'image': {'small': 'https://static.qobuz.com/images/covers/52/36/0075679933652_230.jpg', 'thumbnail': 'https://static.qobuz.com/images/cov

The standard metadata for the track is listed below:

In [22]:
print(f"Track: {track['track_number']}/{track['album']['tracks_count']}")
print(f"Disc: {track['media_number']}/{track['album']['media_count']}")
print(f"Title: {track['title']}")
print(f"Artist: {track['performer']['name']}")
print(f"Album: {track['album']['title']}")
print(f"Album artist: {track['album']['artist']['name']}")
print(f"Genre: {track['album']['genre']['name']}")
print(f"Release date: {track['streamable_at']}")
print(f"Copyright: {track['copyright']}")
print(f"ISRC: {track['isrc']}")

Track: 4/13
Disc: 1/1
Title: Runaway (U & I)
Artist: Galantis
Album: Pharmacy
Album artist: Galantis
Genre: Dance
Release date: 1685170800
Copyright: © 2015 Atlantic Recording Corporation for the United States and WEA International for the world outside of the United States. A Warner Music Group Company ℗ 2014 Atlantic Recording Corporation
ISRC: USAT21404266


Note that `track["performer"]` only contains information about the primary artist associated with the track and can be incomplete. The full list of artists can be found in the track credits:

In [23]:
print(" -", track["performers"].replace(" - ", "\n - "))

 - Cathy Dennis, Lead Vocals
 - Kevin Grainger, MasteringEngineer
 - Christian Karlsson, Writer
 - Niklas Flyckt, MixingEngineer
 - Linus Eklöw, Writer
 - Julia Karlsson, Lead Vocals, Writer
 - Catherine Dennis, Writer
 - Galantis, MainArtist
 - Jimmy Koitzsch, Writer
 - Christian 'Bloodshy' Karlsson, Producer, Instruments, MixingEngineer
 - Anton Rundberg, Programmer, Writer
 - Jimmy 'Svidden' Koitzsch, Producer
 - Linus 'Style Of Eye' Eklöw, Producer, Instruments, MixingEngineer


The track credits, categorized by role, can be obtained by passing `performers=track["performers"]` (or `track_id` directly) to `qobuz.Session.get_track_credits()`:

In [24]:
for key, value in session.get_track_credits(performers=track["performers"]).items():
    print(key, value, sep=": ")

instruments: ["Christian 'Bloodshy' Karlsson", "Linus 'Style Of Eye' Eklöw"]
programmer: ['Anton Rundberg']
lead_vocals: ['Cathy Dennis', 'Julia Karlsson']
mastering_engineer: ['Kevin Grainger']
writer: ['Christian Karlsson', 'Linus Eklöw', 'Julia Karlsson', 'Catherine Dennis', 'Jimmy Koitzsch', 'Anton Rundberg']
mixing_engineer: ['Niklas Flyckt', "Christian 'Bloodshy' Karlsson", "Linus 'Style Of Eye' Eklöw"]
main_artist: ['Galantis']
producer: ["Christian 'Bloodshy' Karlsson", "Jimmy 'Svidden' Koitzsch", "Linus 'Style Of Eye' Eklöw"]


For streaming purposes, the file URL for the track's audio stream can be retrieved using `qobuz.Session.get_track_file_url()`. The Qobuz ID is the first (required) positional argument. The audio quality of the audio stream can optionally be specified using the `quality` argument, with the following possible values:

* `5` or `"LOSSY"` for constant bitrate (320 kbps) MP3
* `6` or `"CD"` for CD-quality (16-bit, 44.1 kHz) FLAC
* `7` or `"HI-RES-96"` for up to 24-bit, 96 kHz Hi-Res FLAC
* default: `27` or `"HI-RES-192"` for up to 24-bit, 192 kHz Hi-Res FLAC

In [25]:
file_url = session.get_track_file_url(track_id)
file_url["url"] = f"{file_url['url'].split('&app_id')[0]}..." # Remove identifying information.
for key, value in file_url.items():
    print(key, value, sep=": ")

track_id: 23929516
duration: 30
url: https://streaming-qobuz-std.akamaized.net/file?eid=23929516&fmt=7&profile=raw&range=20-30...
format_id: 7
mime_type: audio/flac
sample: True
restrictions: [{'code': 'UserUnauthenticated'}]
sampling_rate: 96
bit_depth: 24


For applications where the raw audio stream in `bytes` is necessary, we can use `qobuz.Session.get_track_stream()` with the same arguments instead:

In [26]:
ipywidgets.Audio(value=session.get_track_stream(track_id), format="flac", autoplay=False)

Audio(value=b'fLaC\x00\x00\x00"\x12\x00\x12\x00\x00@Q\x00]\xbb\x17p\x03p\x00+\xf2\x00\x00\x00\x00\x00\x00\x00\…

### Object-oriented approach

First, we directly create a `qobuz.Track` object using the track's Qobuz ID. Information about the track is automatically retrieved.

In [27]:
track = qobuz.Track(track_id, authenticate=False)
print(track)

Qobuz track: 'Runaway (U & I)' [artist: Galantis, album: Pharmacy]


Like before, the standard metadata for the track is listed below:

In [28]:
print(f"Track: {track.track_number}/{track.album.track_count}")
print(f"Disc: {track.disc_number}/{track.album.disc_count}")
print(f"Title: {track.title}")
print(f"Artist: {track.main_artist.name}")
print(f"Album: {track.album.title}")
print(f"Album artist: {track.album.main_artist.name}")
print(f"Genre: {track.album.genre}")
print(f"Release date: {track.release_date}")
print(f"Copyright: {track.copyright}")
print(f"ISRC: {track.isrc}")

Track: 4/13
Disc: 1/1
Title: Runaway (U & I)
Artist: Galantis
Album: Pharmacy
Album artist: Galantis
Genre: Dance
Release date: 2023-05-27 07:00:00
Copyright: © 2015 Atlantic Recording Corporation for the United States and WEA International for the world outside of the United States. A Warner Music Group Company ℗ 2014 Atlantic Recording Corporation
ISRC: USAT21404266


The track credits is only available pre-formatted:

In [29]:
for key, value in track.credits.items():
    print(key, value, sep=": ")

instruments: ["Christian 'Bloodshy' Karlsson", "Linus 'Style Of Eye' Eklöw"]
programmer: ['Anton Rundberg']
lead_vocals: ['Cathy Dennis', 'Julia Karlsson']
mastering_engineer: ['Kevin Grainger']
writer: ['Christian Karlsson', 'Linus Eklöw', 'Julia Karlsson', 'Catherine Dennis', 'Jimmy Koitzsch', 'Anton Rundberg']
mixing_engineer: ['Niklas Flyckt', "Christian 'Bloodshy' Karlsson", "Linus 'Style Of Eye' Eklöw"]
main_artist: ['Galantis']
producer: ["Christian 'Bloodshy' Karlsson", "Jimmy 'Svidden' Koitzsch", "Linus 'Style Of Eye' Eklöw"]


Finally, the file URL and stream data can be obtained by calling methods innate to the `qobuz.Track` object:

In [30]:
track.get_file_url()
track.file_url = f"{track.file_url.split('&app_id')[0]}..." # Remove identifying information.
print(track.file_url, sep="")

https://streaming-qobuz-std.akamaized.net/file?eid=23929516&fmt=7&profile=raw&range=20-30...


In [31]:
track.get_stream()
ipywidgets.Audio(value=track.stream, format="flac", autoplay=False)

Audio(value=b'fLaC\x00\x00\x00"\x12\x00\x12\x00\x00@Q\x00]\xbb\x17p\x03p\x00+\xf2\x00\x00\x00\x00\x00\x00\x00\…

## Working with albums

Albums, including extended plays (EPs) and singles, are collections of tracks by a primary artist. Every track belongs to an album.

In the following section, we will retrieve information about an album (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will use the album 'Brighter Days' by Sigala.

In [32]:
album_id = "i39dkeqnl9e9b"

### Procedural approach

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the album:

In [33]:
session = qobuz.Session(authenticate=False)
album = session.get_album(album_id)
for key, value in album.items():
    print(key, value, sep=": ")

maximum_bit_depth: 24
image: {'small': 'https://static.qobuz.com/images/covers/9b/9e/i39dkeqnl9e9b_230.jpg', 'thumbnail': 'https://static.qobuz.com/images/covers/9b/9e/i39dkeqnl9e9b_50.jpg', 'large': 'https://static.qobuz.com/images/covers/9b/9e/i39dkeqnl9e9b_600.jpg', 'back': None}
media_count: 1
artist: {'image': None, 'name': 'Sigala', 'id': 2199202, 'albums_count': 127, 'slug': 'sigala', 'picture': None}
artists: [{'id': 2199202, 'name': 'Sigala', 'roles': ['main-artist']}]
upc: 0886446794235
released_at: 1538085600
label: {'name': 'Ministry of Sound Recordings', 'id': 151038, 'albums_count': 1341, 'supplier_id': 23, 'slug': 'ministry-of-sound-recordings'}
title: Brighter Days
qobuz_id: 50586621
version: None
url: https://www.qobuz.com/fr-fr/album/brighter-days-sigala/i39dkeqnl9e9b
duration: 3379
popularity: 0
tracks_count: 16
genre: {'path': [64, 129], 'color': '#5eabc1', 'name': 'Dance', 'id': 129, 'slug': 'dance'}
maximum_channel_count: 2
id: i39dkeqnl9e9b
maximum_sampling_rate: